In [1]:
import numpy as np

In [2]:
with open('berlin_coordinates.npy', 'rb') as f:
    coordinate = np.load(f)

In [3]:
!pip install routingpy

In [4]:
#setup OSRM 
from routingpy import OSRM
from routingpy.routers import options
options.default_timeout=None
from sklearn.preprocessing import normalize
from torch.utils.data import DataLoader, Dataset
import torch
import random

client = OSRM(base_url="https://router.project-osrm.org")

In [5]:
# Testing

def generate(coordinate, graphsize=10, num_sample=100000, batch=1000):
    n = graphsize + 1
    rng = np.random.default_rng(1235)

    
    #demand = rng.randint(1,10,[num_sample,n,1])
    # Set the demand of depot to true
    #demand[:,0]=0
    #data = np.concatenate([instances,demand],2)

    client = OSRM(base_url="https://router.project-osrm.org")
    # instead of looping through the instances/coordinates
    # try to generate 10k or 15k examples at once
    # Such that if we are required to generate 500k instance we will do that 100 times
    # We can generate let say 50k but I want we use all the 835 coordinate
    # After every batch, we will sample a new 100 random coordinates
    graphs = []
    demands = []
    distances= []
    times = []
    for i in range(0, num_sample, batch):

        # Sample 100 random coordinates
        coord_len = len(coordinate)
        coord_idx = np.random.choice(coord_len, size=(100), replace=True)
        coord_100 = coordinate[coord_idx]
        #coord = np.flip(coord_100) 
        idx = list(range(len(coord_100)))
        
        dist_matrix = client.matrix(coord_100, profile="car", sources= idx[0] ,destinations= idx)
        if np.any(dist_matrix.distances):
            pass
        else:
            dist_matrix = client.matrix(coord_100, profile="car", sources= idx[0] ,destinations= idx)
            
        graph_size_idx = rng.choice(idx, size=(batch,n), replace=True)

        # Get graph coordinates
        graph = coord_100[graph_size_idx]
        graphs.append(graph)

        print(np.array(dist_matrix.distances).shape)

        # Get distance and time matrix
        col = np.full((batch,n,n), graph_size_idx.reshape(batch,-1,n))
        #print(type(dist_matrix.distances))
        row = np.transpose(col, (0,2,1))
        distances.append(np.array(dist_matrix.distances)[row,col])
        times.append(np.array(dist_matrix.durations)[row,col])

        # Get demand
        demand = np.random.randint(low=10, high=50, size=(batch,n))
        # set depot to zero
        demand[:,0] = 0
        demands.append(demand)

        # dist_matrix returns 100 x 100 
        #distance.append(dist_matrix.distances)
        #duration.append(dist_matrix.durations)
        
        # Now we will generate the first 10k batch
        #Since we have 100 cordinates, Sample 10k by n idx
        # To be able to index distance matrix, we need indices for row and col
        # These indices will be of the shape 10k x n x n
        # Such that the returned distance matrix is  10k x n x n 
        # Futher more we will get the cordinate 10k x n
        # The demand 10k x n, the depot has a demand of zero
     
    with open(f'dataset_{num_sample}_{graphsize}.npy', 'wb') as f:
        np.save(f, np.concatenate(graphs))
        np.save(f, np.concatenate(demands))
        np.save(f, np.concatenate(distances))
        np.save(f, np.concatenate(times))

    


    return (graphs, demands, distances)



In [ ]:
generate(coordinate, graphsize=10, num_sample=100000, batch=1000)

In [18]:
class VRPDataset(Dataset):
    def __init__(self, path):
        super().__init__()
        self._graphs,self._demands, self._distances, self._time = VRPDataset.open_file(path)
        _,self.nrow, _ = self._graphs.shape
    
    def __len__(self):
        return len(self._graphs)

    def __getitem__(self, idx):
        nrow, _ = self._graphs[idx].shape
        padding = np.ones((nrow, 2))
        # Indicates if a node is a depot or not
        
        padding[1:,0] = 0
        padding[0,1] = 0
        # Preprocessing
        demand = self._demands[idx]
        demand = normalize(demand.reshape(1,-1))
        graph = normalize(self._graphs[idx])
        node_features = np.concatenate([padding,demand.T,graph], axis=1)
        edge_features = np.divide(self._distances[idx],1000)
        #data = self.vrpdataset[index]
        #node_features = data[:20] # To be changed
        #edge_features = data[20:] # To Do
        return {
            'node_features':torch.from_numpy(node_features),
            'edge_features':torch.from_numpy(edge_features),
            'coordinates':torch.from_numpy(self._graphs[idx])

        }


        
    @staticmethod
    def open_file(path):
        with open(path, 'rb') as f:
            graphs = np.load(f)
            demands = np.load(f)
            distances = np.load(f)
            times = np.load(f)
        return graphs, demands, distances,times


In [19]:
path = 'dataset_200_10.npy'
data = VRPDataset(path)

In [23]:
train_dataloader = DataLoader(data, batch_size=64, shuffle=True)
